# 02 - Baseline Models

In [5]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import numpy as np
import pandas as pd
import os
import sys
import pprint

import matplotlib.pyplot as plt
import matplotlib as mpl

import seaborn as sns
plt.style.use('ggplot')
sns.set()
# plt.style.use('seaborn')
mpl.style.use('seaborn')

pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_columns = 999
pd.options.display.max_rows = 500
np.set_printoptions(precision=6)

DATA_PATH = '../data'
pp = pprint.PrettyPrinter(indent=4)

print(pd.__version__)

# UGLY HACK - not for production
sys.path.append('..')

0.23.4


In [40]:
import utils

TRAIN_FILE = os.path.join(DATA_PATH, 'processed', 'baseline', 'train.csv')
VALID_FILE = os.path.join(DATA_PATH, 'processed', 'baseline', 'val.csv')

train_df = pd.read_csv(TRAIN_FILE)
valid_df = pd.read_csv(VALID_FILE)

print(train_df.shape)
print(valid_df.shape)

FEATURES_FILE = os.path.join(DATA_PATH, 'processed', 'baseline', 'features.json')
features = utils.Features()
features.load(FEATURES_FILE)

# pp.pprint(features)

continuous = features['continuous']
output = features['output']

print(continuous)

(10066, 28)
(1258, 28)
['age', 'time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']


In [43]:
print(train_df['readmitted'].value_counts())
print(valid_df['readmitted'].value_counts())

1    5033
0    5033
Name: readmitted, dtype: int64
1    629
0    629
Name: readmitted, dtype: int64


In [30]:
X_train = train_df[continuous].values.astype(np.float32)
X_valid = valid_df[continuous].values.astype(np.float32)

X_train = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0)
X_valid = (X_valid - X_train.mean(axis=0)) / X_train.std(axis=0)

y_train = train_df[output].values.astype(np.float32)
y_valid = valid_df[output].values.astype(np.float32)
y_train = y_train.ravel()
y_valid = y_valid.ravel()

# print(np.shape(X_train))
# print(np.shape(y_train))


In [29]:
from sklearn.svm import SVC

svc = SVC(probability=True, kernel='rbf')
svc.fit(X_train, y_train.ravel())

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [34]:
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

output_train = svc.predict(X_train)
target_train = y_train

output_valid = svc.predict(X_valid)
target_valid = y_valid


In [36]:
acc_train = accuracy_score(target_train, output_train)
auc_train = roc_auc_score(target_train, output_train)

acc_valid = accuracy_score(target_valid, output_valid)
auc_valid = roc_auc_score(target_valid, output_valid)

print('Train')
print(f'  Acc: {acc_train}')
print(f'  Auc: {auc_train}')
print(confusion_matrix(target_train, output_train))

print('Valid')
print(f'  Acc: {acc_valid}')
print(f'  Auc: {auc_valid}')
print(confusion_matrix(target_valid, output_valid))

Train
  Acc: 0.5952712100139083
  Auc: 0.5952712100139083
[[3181 1852]
 [2222 2811]]
Valid
  Acc: 0.5015898251192369
  Auc: 0.5015898251192369
[[  2 627]
 [  0 629]]
